### Import libs

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path

### Declarate URL and PATH

In [ ]:
URL = "https://www.ipeadata.gov.br/ExibeSerieR.aspx?stub=1&serid=2096726409&MINDATA=1970&MAXDATA=2030&TNIVID=2&TPAID=1&module=S"


OUT_CSV = Path("taxa_analfabetismo_2022_por_uf.csv") 


###  Making Request

In [16]:
html = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"}).text

### Extract principal data

In [17]:
soup  = BeautifulSoup(html, "lxml")
rows  = soup.select("table#grd_DXMainTable tr[id^=grd_DXDataRow]")

### Convert to list of dictionaries

In [ ]:
records = []
for tr in rows:
    cols = [td.get_text(strip=True).replace("\xa0", " ") for td in tr.find_all("td")]
    sigla, estado, v1991, v2000, v2010, v2022 = cols
    records.append({
        "UF":     sigla,
        "Estado": estado,
        "1991":   float(v1991.replace(",", ".")),
        "2000":   float(v2000.replace(",", ".")),
        "2010":   float(v2010.replace(",", ".")),
        "2022":   float(v2022.replace(",", "."))
    })

ValueError: too many values to unpack (expected 6)

### Create Dataframe

In [7]:
df = pd.DataFrame(records).set_index("UF")

### Save Data

In [11]:
df[["Estado", "2022"]].sort_values("2022").to_csv("/home/fbotero/Documents/POS/raw/analfabetismo_uf_2022.csv", float_format="%.2f", encoding="utf-8")